In [1]:
import pandas as pd
import numpy as np
from utils import convert
from config import api_key, secret_key
import time
from binance.client import Client
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

Объединим все части нашей программы
Чтобы процесс проходил следующим образом:

    1. Получаю данные за 15 последних минут из client.get_klines()
    
    2. Преобразую их в нужный вид для входа в нейросеть с помощью функции convert
    
    3. Подаю данные в нейросеть и получаю ответ
    
    4. В зависимости от ответа, покупаю, либо ничего не делаю и жду следующую минуту

Но, для начала надо, чтобы некоторые вещи были подготовлены:

    1. Был обучен scaler
    
    2. Была определена нейронная сеть(через сохраненные веса после оубчения либо через сохраненную модель)

In [2]:
#загружаем модель с обученными весами
model = load_model("57_test_precision.h5")

In [3]:
#обучаем scaler
data_for_fit_scaler = pd.read_csv("df_to_fit_scaler.csv")
scaler = MinMaxScaler()
scaler.fit(data_for_fit_scaler)

MinMaxScaler()

Начинаем соединять все части программы

In [4]:
#получаем один экземпляр на минутном таймфрейме
asset = "BTCUSDT"
client = Client(api_key = api_key, api_secret = secret_key)
sample = client.get_klines(symbol = asset, interval = "1m", limit = 15)

In [5]:
#конвертируем
convert_sample = convert(sample, scaler)

In [6]:
#узнаем результат
result = model.predict(convert_sample)

In [7]:
print(result)

[[0.0123848]]


In [8]:
#напишем простенький скрипт с объединением этих этапов
import time
import datetime

def main_script():
    sample = client.get_klines(symbol = asset, interval = "1m", limit = 15)
    convert_sample = convert(sample, scaler)
    result = model.predict(convert_sample)
    time_curr = datetime.datetime.fromtimestamp(time.time()).minute + 1
    if result >0.5:
        print(f"На {time_curr} минуте должен быть скачек на 0.5")
    else:
        print(f"на {time_curr} ничего нет")

In [10]:
#затестим на сеюминутных данных данных
for i in range(20):
    main_script()
    time.sleep(60)

на 2 ничего нет
на 3 ничего нет
на 4 ничего нет
на 5 ничего нет
на 6 ничего нет
на 7 ничего нет
на 8 ничего нет
на 9 ничего нет
на 10 ничего нет
на 11 ничего нет
на 12 ничего нет
на 13 ничего нет
на 14 ничего нет
на 15 ничего нет
на 16 ничего нет
на 17 ничего нет
на 18 ничего нет
на 19 ничего нет
на 20 ничего нет
на 21 ничего нет
